In [7]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from datetime import datetime
from pyspark.sql import types
from collections import namedtuple
import pandas as pd

In [8]:
#  gcloud auth application-default login
cred = "/home/user/.config/gcloud/application_default_credentials.json"

conf = ( 
    SparkConf()
    .setMaster("local[*]")
    .setAppName('test') 
    .set("spark.jars", "./lib/gcs-connector-hadoop3-2.2.5.jar") 
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") 
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", cred)
)

In [3]:
sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", cred)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

23/04/03 15:06:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
spark = SparkSession.builder.config(conf=sc.getConf()).getOrCreate()

In [5]:
spark.sparkContext.uiWebUrl

'http://dtc-de.asia-southeast1-b.c.de-dtc-375915.internal:4040'

In [6]:
df_green = spark.read.parquet('gs://prefect-dtc/green/*')

In [9]:
df_green.show()


+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2018-12-21 15:17:29|  2018-12-21 15:18:57|                 N|         1|         264|         264|              5|          0.0|        3.0|  0.5|    0.